In [ ]:
# Pip installs
!pip install git+https://github.com/johnwmillr/LyricsGenius.git
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
import evaluate
!pip install -q rouge_score
#let's make longer output readable without horizontal scrolling
from pprint import pprint

  Cloning https://github.com/johnwmillr/LyricsGenius.git to /tmp/pip-req-build-j865xau5
  Running command git clone --filter=blob:none --quiet https://github.com/johnwmillr/LyricsGenius.git /tmp/pip-req-build-j865xau5
  Resolved https://github.com/johnwmillr/LyricsGenius.git to commit ee6c7fbb826dcbd99d5a75c49fde3a55bee43014
  Preparing metadata (setup.py) ... done
  Created wheel for lyricsgenius: filename=lyricsgenius-3.0.1-py3-none-any.whl size=44769 sha256=9082f41ba46a2d98a652066218bd6b48db5526b1a0a230411be742b0254b412f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z73qcttk/wheels/a2/6d/80/1297c611c6dc8bd51f83121a1e209257591bc9a0486bd2e279
Successfully built lyricsgenius
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0

In [ ]:
# imports
import requests
from tabulate import tabulate
import csv
import lyricsgenius
import pandas as pd
import re

In [ ]:
# Genius setup
# Sahana's API Token
GENIUS_API_TOKEN = 'gYStp8V74DgB-WG7kAjtlySn5lJviczVv6pG9Sp1qpjf05MUb8yJIHvfZUPG9UaP'
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

# Set up the base URL and headers for the Genius API
base_url = "https://api.genius.com"
headers = {'Authorization': f'Bearer {GENIUS_API_TOKEN}'}

In [ ]:
import csv
def export_song_data_to_csv(song_data_list, artist_name):
    # Define all fieldnames, ensuring "Lyrics" is included
    fieldnames = ['Song ID', 'Title', 'Lyrics URL', 'Combined Annotations', 'Wikipedia Annotation', 'Lyrics']
    filename = f"{artist_name}_top_50_songs.csv"

    # Write data to CSV
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()  # Write header row
        writer.writerows(song_data_list)  # Write each row from the song data list

In [ ]:
# Step 1: Function to search for an artist and retrieve the artist ID
def get_artist_id(artist_name):
    search_url = f"{base_url}/search"
    params = {'q': artist_name}

    # Make a request to search for the artist
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
        results = response.json()
        hits = results['response']['hits']

        # Look for the first matching artist in the search results
        for hit in hits:
            if hit['result']['primary_artist']['name'].lower() == artist_name.lower():
                return hit['result']['primary_artist']['id']

    return None

In [ ]:
# Step 2: Function to get all songs by artist ID, but only those with annotations
def get_songs_with_annotations_by_artist_id(artist_id, per_page=50, max_songs=200):
    songs_with_annotations = []
    page = 1

    # Keep fetching songs until we hit the limit or run out of pages
    while len(songs_with_annotations) < max_songs:
        songs_url = f"{base_url}/artists/{artist_id}/songs"
        params = {
            'page': page,
            'per_page': per_page
        }

        response = requests.get(songs_url, headers=headers, params=params)

        if response.status_code == 200:
            song_data = response.json()
            new_songs = song_data['response']['songs']
            if not new_songs:
                break  # Stop if no more songs are returned

            # Filter only songs with annotations
            for song in new_songs:
                if song['annotation_count'] > 0:
                    songs_with_annotations.append(song)

            page += 1
        else:
            break

    return songs_with_annotations

In [ ]:
# Step 3: Function to get annotations for a song using the song ID and fetch song URL
def get_song_annotations_and_url(song_id):
    song_url = f"{base_url}/songs/{song_id}"

    # Make a request to get the song data
    response = requests.get(song_url, headers=headers)

    if response.status_code == 200:
        song_data = response.json()['response']['song']
        annotations = []
        lyrics_url = song_data.get('url', 'N/A')  # Fetch the lyrics URL

        # Fetch annotations from the song description if available
        if 'description' in song_data and song_data['description']['dom']:
            dom_annotations = song_data['description']['dom']['children']
            for annotation in dom_annotations:
                if isinstance(annotation, str):  # Plain text annotations
                    annotations.append(annotation)
                elif 'children' in annotation:  # Complex annotations
                    for child in annotation['children']:
                        if isinstance(child, str):
                            annotations.append(child)

        # Combine all annotations into one string
        combined_annotations = " ".join(annotations)

        return lyrics_url, combined_annotations
    else:
        print("Failed to fetch song data.")
        return None, None

In [ ]:
# Define the get_lyrics function to fetch lyrics from Lyrics.ovh API
def get_lyrics(artist_name, song_title):
    url = f"https://api.lyrics.ovh/v1/{artist_name}/{song_title}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            lyrics = data.get("lyrics")
            if lyrics and lyrics.strip():
                return lyrics
        return None
    except:
        return None

In [ ]:
import requests

# Function to fetch Wikipedia annotation for a given song title
def get_wikipedia_annotation(song_title):
    search_url = "https://en.wikipedia.org/w/api.php"
    search_params = {
        "action": "opensearch",
        "format": "json",
        "search": song_title
    }

    response = requests.get(search_url, params=search_params)
    search_data = response.json()

    # Check if we have results
    if len(search_data[1]) > 0:
        # Fetch the first search result
        page_title = search_data[1][0]
        page_url = "https://en.wikipedia.org/w/api.php"
        page_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts",
            "titles": page_title,
            "explaintext": True  # Fetch plain text for easy processing
        }
        page_response = requests.get(page_url, params=page_params)
        page_data = page_response.json()
        pages = page_data.get("query", {}).get("pages", {})

        # Extract the page content
        for page_id, page_content in pages.items():
            return page_content.get("extract", "No Wikipedia annotation found")

    return "No Wikipedia annotation found"


In [ ]:
from tabulate import tabulate
import re
import time
import pandas as pd

def get_top_50_songs_with_annotations_by_artist(artist_name):
    # Get artist ID using the existing function
    artist_id = get_artist_id(artist_name)

    if artist_id:
        # Get songs with annotations using existing function
        songs = get_songs_with_annotations_by_artist_id(artist_id)

        if songs:
            # Fetch annotations, URLs, and Wikipedia annotations
            song_data_list = []
            for song in songs:
                song_id = song['id']
                lyrics = get_lyrics(artist_name, song['title'])

                # Only process songs that have lyrics
                if lyrics:
                    lyrics_url, combined_annotations = get_song_annotations_and_url(song_id)
                    wikipedia_annotation = get_wikipedia_annotation(song['title'])

                    song_data_list.append({
                        'Song ID': song_id,
                        'Title': song['title'],
                        'Lyrics URL': lyrics_url,
                        'Combined Annotations': combined_annotations,
                        'Wikipedia Annotation': wikipedia_annotation,
                        'Lyrics': lyrics
                    })

            # Sort after collecting all songs with lyrics
            sorted_songs = sorted(song_data_list, key=lambda x: (-len(x['Combined Annotations']), x['Title']))
            top_50_songs = sorted_songs[:50]

            # Filter Wikipedia annotations silently
            for song_data in top_50_songs:
                if artist_name.lower() not in song_data['Wikipedia Annotation'].lower():
                    song_data['Wikipedia Annotation'] = "No Wikipedia annotation found (artist name not mentioned)"

            return top_50_songs

    return None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# List of all artists
from google.colab import drive
drive.mount('/content/drive')

# The folder ID "10Fevi_myWRyxiT6waMFhoPANW9r1jrCg"
folder_id = "10Fevi_myWRyxiT6waMFhoPANW9r1jrCg"
save_folder = f"/content/drive/MyDrive/genius_lyrics_outputs"

# Create the folder if it doesn't exist
import os
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
#done with "Morgan Wallen", "SZA", "Taylor Swift", "Drake", "Luke Combs", Miley
artist_names = ["Justin Timberlake", "AC/DC", "Metallica",
    "The Beach Boys", "Seventeen", "The Red Clay Strays", "Creedence Clearwater Revival",
    "Crosby, Stills, Nash & Young", "Linkin Park", "Green Day", "Maroon 5", "Black Eyed Peas", "ABBA",
]

# Process each artist
total_artists = len(artist_names)
for index, artist_name in enumerate(artist_names, 1):
    print(f"Processing {artist_name} ({index}/{total_artists})...", end=" ", flush=True)

    try:
        songs_data = get_top_50_songs_with_annotations_by_artist(artist_name)

        if songs_data:
            safe_filename = re.sub(r'[^\w\s-]', '', artist_name).replace(' ', '_')
            csv_filename = os.path.join(save_folder, f"{safe_filename}_top_50_songs.csv")

            df = pd.DataFrame(songs_data)
            df.to_csv(csv_filename, index=False, encoding='utf-8')
            print(f"Done! ({len(songs_data)} songs)")
        else:
            print("No data found")

    except Exception as e:
        print(f"Error: {str(e)}")
        error_log_path = os.path.join(save_folder, 'error_log.txt')
        with open(error_log_path, 'a') as f:
            f.write(f"{artist_name}: {str(e)}\n")
        continue

    time.sleep(2)

print("\nAll processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing Justin Timberlake (1/13)... Done! (50 songs)
Processing AC/DC (2/13)... No data found
Processing Metallica (3/13)... Done! (50 songs)
Processing The Beach Boys (4/13)... Done! (43 songs)
Processing Seventeen (5/13)... No data found
Processing The Red Clay Strays (6/13)... No data found
Processing Creedence Clearwater Revival (7/13)... Done! (50 songs)
Processing Crosby, Stills, Nash & Young (8/13)... No data found
Processing Linkin Park (9/13)... Done! (50 songs)
Processing Green Day (10/13)... Done! (50 songs)
Processing Maroon 5 (11/13)... Done! (50 songs)
Processing Black Eyed Peas (12/13)... Error: 'children'
Processing ABBA (13/13)... Done! (50 songs)

All processing complete!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
taylor = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Taylor Swift_top_50_songs_with_annotations.csv')
taylor.head()

,Song ID,Title,Lyrics URL,Combined Annotations,Lyrics
0,3071324,Artist Archive: Taylor Swift,https://genius.com/Genius-artist-archive-taylo...,Welcome to the Artist Archive! The goal of t...,2021 to Present: Taylor's Version Era\nAwards ...
1,5264451,Colors in Taylor Swift Songs,https://genius.com/Genius-users-colors-in-tayl...,"Throughout her decade-spanning career, has o...","Amber- ""marjorie""Aquamarine- ""Slut!""Black- ""ca..."
2,7076626,All Too Well (10 Minute Version) (Taylor’s Ver...,https://genius.com/Taylor-swift-all-too-well-1...,"Widely regarded as one of Swift’s , “All To...","[Verse 1]\nI walked through the door with you,..."
3,1862691,“Bad Blood” Video Breakdown,https://genius.com/Taylor-swift-bad-blood-vide...,Watch the “Bad Blood” music video here! Dire...,Cold Open \n\n\nThe Betrayal \n\n\nSmashed Car...
4,7394358,All Too Well (10 Minute Version) (Taylor’s Ver...,https://genius.com/Taylor-swift-all-too-well-1...,Swift’s release of —as a part of her 2021 rer...,"[Verse 1]\nI walked through the door with you,..."


In [ ]:
!ls /content/drive/MyDrive

'266 Final Project'   Jack_Lucas_Chang_Resume_Lyft.pdf
'Colab Notebooks'    'MISC Classes'
 DATASCI201	     'Report for Emerging Ethical Tech Workforce.gdoc'
'ischool thigs'      'Resume Career'


In [ ]:
# Function to clean up the text
print(taylor['Combined Annotations'])

def clean_text(text):
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove incomplete sentences (e.g., "   ")
    text = re.sub(r'(\s*—\s*)', '', text)  # Handle em-dash
    text = re.sub(r'\s+[,\.]', '', text)  # Remove space before punctuation
    text = re.sub(r'[“”]', '"', text)  # Replace smart quotes
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s{2,}', ' ', text)  # Remove extra spaces again

    # Optionally, further processing can be added for placeholder gaps (e.g., '   ')
    # For now, we can replace any leftover placeholders with a generic "[missing]" or remove them.
    text = re.sub(r'\s{2,}', '[missing]', text)

    # Strip leading/trailing whitespace
    text = text.strip()

    return text

taylor['Combined Annotations'] = taylor['Combined Annotations'].apply(clean_text)

taylor.head()

0     Welcome to the   Artist Archive! The goal of t...
1     Throughout her decade-spanning career,   has o...
2     Widely regarded as one of Swift’s    , “All To...
3     Watch the “Bad Blood” music video here!   Dire...
4     Swift’s release of  —as a part of her 2021 rer...
5     In ‘Cassandra,’ Taylor Swift explores themes o...
6     “Clara Bow” is the last track on eleventh stud...
7     “Chloe or Sam or Sophia or Marcus” tells the s...
8     Widely regarded as one of Swift’s    , “All To...
9                                                     ?
10                                                    ?
11    Accompanied by the release of her project writ...
12    On the day of her   appearance to talk about t...
13    “But Daddy I Love Him” is the sixth track on T...
14    “Cornelia Street” focuses on the central role ...
15    A day before   was released,     the release o...
16    “champagne problems” tells the story of a woma...
17    The liner notes for her fifth studio album

,Song ID,Title,Lyrics URL,Combined Annotations,Lyrics
0,3071324,Artist Archive: Taylor Swift,https://genius.com/Genius-artist-archive-taylo...,Welcome to the Artist Archive! The goal of thi...,2021 to Present: Taylor's Version Era\nAwards ...
1,5264451,Colors in Taylor Swift Songs,https://genius.com/Genius-users-colors-in-tayl...,"Throughout her decadespanning career, has ofte...","Amber- ""marjorie""Aquamarine- ""Slut!""Black- ""ca..."
2,7076626,All Too Well (10 Minute Version) (Taylor’s Ver...,https://genius.com/Taylor-swift-all-too-well-1...,Widely regarded as one of Swifts All Too Well ...,"[Verse 1]\nI walked through the door with you,..."
3,1862691,“Bad Blood” Video Breakdown,https://genius.com/Taylor-swift-bad-blood-vide...,Watch the Bad Blood music video here! Directed...,Cold Open \n\n\nThe Betrayal \n\n\nSmashed Car...
4,7394358,All Too Well (10 Minute Version) (Taylor’s Ver...,https://genius.com/Taylor-swift-all-too-well-1...,Swifts release ofas a part of her 2021 rerelea...,"[Verse 1]\nI walked through the door with you,..."


In [ ]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
t5model.summary()

In [ ]:
# Using one as an example, otherwise we would loop
ARTICLE_TO_SUMMARIZE = taylor.iloc[2]['Combined Annotations']

In [ ]:
# Let's determine the min/max length
#######
# For this very specific example
print(len(taylor.iloc[2]['Combined Annotations']))
print(taylor.iloc[2]['Combined Annotations'])
#######
# Otherwise in general
# Calculate the length of each value in the 'Combined Annotations' column
taylor['annotation_length'] = taylor['Combined Annotations'].apply(len)

# Calculate the average length
average_length = taylor['annotation_length'].mean()

# Display the result
print(f"Average length of 'Combined Annotations': {average_length}")

1801
Widely regarded as one of Swifts All Too Well sees her picking up shards of memories after a gutwrenching breakup. She jumps through time to reflect on her best and worst memories with her ex, making the listener feel as though they were there with the couple. Swift told that she began writing the song in a soundcheck for her 2011 tour in support of Cowriter explained that the song was initially before she helped Swift pare it down, and Swift shared some of the in a deluxe edition of her 2019 album, Melodically, this version of All Too Well is more subdued than the version we hear on the album. This change is evident throughout the song. This version begins with a saccharine melody before building up to a steady drum beat, while the original is heavily backed by guitar. Lyrically, the songs are also different. This ten minute version is more vitriolic and bitter, while the album version is bittersweet. The metaphors in Verse 3 are reminiscent of era Taylor, rather than era, with c

In [ ]:
PROMPT = 'summarize: '
T5ARTICLE_TO_SUMMARIZE = PROMPT + ARTICLE_TO_SUMMARIZE
inputs = t5tokenizer(T5ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")
summary_ids = t5model.generate(inputs["input_ids"],
                               max_new_tokens=2000,
                               min_new_tokens=513,
                               no_repeat_ngram_size=3,
                               num_beams=5,
                               early_stopping=True
)
annotation_summary = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(annotation_summary[0], compact=True)

('all too well sees Swift picking up shards of memories after a gutwrenching '
 'breakup . she jumps through time to reflect on her best and worst memories '
 'with her ex . many believe Swift took inspiration from her relationship with '
 'actor maple lattes .')


In [ ]:
# Compare to annotation
pprint(taylor.iloc[2]['Combined Annotations'], compact=True)

('Widely regarded as one of Swifts All Too Well sees her picking up shards of '
 'memories after a gutwrenching breakup. She jumps through time to reflect on '
 'her best and worst memories with her ex, making the listener feel as though '
 'they were there with the couple. Swift told that she began writing the song '
 'in a soundcheck for her 2011 tour in support of Cowriter explained that the '
 'song was initially before she helped Swift pare it down, and Swift shared '
 'some of the in a deluxe edition of her 2019 album, Melodically, this version '
 'of All Too Well is more subdued than the version we hear on the album. This '
 'change is evident throughout the song. This version begins with a saccharine '
 'melody before building up to a steady drum beat, while the original is '
 'heavily backed by guitar. Lyrically, the songs are also different. This ten '
 'minute version is more vitriolic and bitter, while the album version is '
 'bittersweet. The metaphors in Verse 3 are remin

In [ ]:
# Let's use the lyrics
input_text = "summarize: " + taylor.iloc[2]['Lyrics']

# Tokenize the input text
input_ids = t5tokenizer.encode(input_text, return_tensors='tf', max_length=512, truncation=True)

# Generate the summary
summary_ids = t5model.generate(input_ids, max_length=500, num_beams=2, length_penalty=2.0, early_stopping=True)

lyrics_summary = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

pprint(lyrics_summary[0], compact=True)

("pre-christian 'pre-christian' was buried in the middle of the night in the "
 'middle of the night . he was a little kid with glasses in a twin-sized bed '
 "and your mother's telling stories 'bout you on the tee-ball team You taught "
 "me 'bout your past, thinkin' your future was me' 'you taught me 'bout your "
 "past, thinkin' your future was me'")


In [ ]:
# Combine both the lyrics summary and annotation summary and see what happens
combined_summary = annotation_summary[0] + " " + lyrics_summary[0]
pprint(combined_summary, compact=True)

input_text = "summarize: " + combined_summary

# Tokenize the input text
input_ids = t5tokenizer.encode(input_text, return_tensors='tf', max_length=512, truncation=True)

# Generate the summary
summary_ids = t5model.generate(input_ids, max_length=50, num_beams=2, length_penalty=2.0, early_stopping=True)

lyrics_and_annotation_summary = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

pprint(lyrics_and_annotation_summary[0], compact=True)

('all too well sees Swift picking up shards of memories after a gutwrenching '
 'breakup . she jumps through time to reflect on her best and worst memories '
 'with her ex . many believe Swift took inspiration from her relationship with '
 "actor maple lattes . pre-christian 'pre-christian' was buried in the middle "
 'of the night in the middle of the night . he was a little kid with glasses '
 "in a twin-sized bed and your mother's telling stories 'bout you on the "
 "tee-ball team You taught me 'bout your past, thinkin' your future was me' "
 "'you taught me 'bout your past, thinkin' your future was me'")
('all too well sees Swift picking up shards of memories after a gutwrenching '
 'breakup . she jumps through time to reflect on her best and worst memories '
 'with her ex . many believe Swift took inspiration from')


In [ ]:
# What if we combined the raw lyrics and annotations?
input_lyrics_and_annotations = taylor.iloc[2]['Lyrics'] + ' ' + taylor.iloc[2]['Combined Annotations']
input_text = "summarize: " + input_lyrics_and_annotations

# Tokenize the input text
input_ids = t5tokenizer.encode(input_text, return_tensors='tf', max_length=512, truncation=True)

# Generate the summary
summary_ids = t5model.generate(input_ids, max_length=50, num_beams=2, length_penalty=2.0, early_stopping=True)

lyrics_and_annotation_summary = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

pprint(lyrics_and_annotation_summary[0], compact=True)

In [ ]:
#In order to not consume all of the memory available in Colab we'll free up the memory we're using for these large language models
del t5model
del t5tokenizer

Trying the shorter Pegasus model here

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [ ]:
inputs = ptokenizer(ARTICLE_TO_SUMMARIZE, max_length=2000, truncation=True, return_tensors="tf")
# Generate Summary
summary_ids = pmodel.generate(inputs["input_ids"]
)
pprint(ptokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

In [ ]:
del pmodel
del ptokenizer

Trying the longer pegasus model here

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)

cnninputs = cnntokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")

In [ ]:
# Generate Summary
summary_ids = cnnmodel.generate(inputs["input_ids"]
)

pprint(cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

In [ ]:
# Character count
# Good Grammar
# References song lyrics
# Complete sentences

